In [231]:
import openai,os,requests,json,re
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [232]:
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI

In [233]:
url = "https://park.ajinomoto.co.jp/recipe/card/705645/"

In [234]:
initial_str = """\
## Output Format
```json
{output_format}
```

## Input
{text}

## Instruction
[Input]はあるWebページのテキストの一部です。\
[Output Format]に従ってデータを入力してください。\
[Input]から読み取れないデータは出力に含めないでください。
"""

refine_str = """\
## Output Format
```json
{output_format}
```

## Data
{data}

## Text
{text}


## Instruction
あなたのタスクは、最終的なデータを生成することです。
現時点でのデータは[Data]です。
[Text]をもとに、必要な場合のみ[Data]を修正してください。
[Text]が必要な情報を提供しない場合、[Data]をそのまま出力してください。
"""

In [235]:
initial_prompt_template = ChatPromptTemplate.from_template(initial_str)
print('initial:',initial_prompt_template.messages[0].prompt.input_variables)
refine_prompt_template = ChatPromptTemplate.from_template(refine_str)
print('refine:',refine_prompt_template.messages[0].prompt.input_variables)

initial: ['output_format', 'text']
refine: ['data', 'output_format', 'text']


In [236]:
response = requests.get(url)
with open('temp.html','w') as f:
    f.write(response.text)
loader = UnstructuredHTMLLoader("temp.html")

In [237]:
data = loader.load()

In [238]:
# data[0]

In [239]:
content_len = len(data[0].page_content)

In [240]:
content_len

14237

In [241]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 2048,
    chunk_overlap  = 1024,
    length_function = len,
)

In [242]:
docs = text_splitter.split_documents(data)

In [243]:
def extract_json(text,debug=False):
    patterns = [
        {"pattern": r'```json\n(.*?)```', "group": 1},
        {"pattern": r'\{.*\}', "group": 0},
    ]
    for p in patterns:
        match = re.search(p["pattern"], text, re.DOTALL)
        if match:
            json_string = match.group(p["group"])
            # print(json_string)
            recipe_json = json.loads(json_string)
            return recipe_json
    print('Error:Could not json extracted:',text)
    return {}

In [244]:
recipe_format = {
    "name": "レシピ名",
    "description": "レシピの説明",
    "cooking_time": "int: minutes",
    "appliances": [
        "str: 調理器具"
    ],
    "serves": "int: 人前",
    "ingredients": [
        {
            "name": "str: 材料名",
            "amount": "str: 分量表記",
            "g": "int: 分量表記から推定した重量(gram)"
        }
    ],
    "steps": {
        "str: 手順番号":"str: 手順の説明"
    }
}

In [245]:
chat = ChatOpenAI(temperature=0.0)

In [246]:
len(docs)

12

In [248]:
recipe = {}
for i,doc in enumerate(docs):
    print(i+1,'-'*30)
    print(doc.page_content)
    text = doc.page_content
    # if i == 0:
    #     messages = initial_prompt_template.format_messages(
    #         output_format=json.dumps(recipe_format,indent=2,ensure_ascii=False),
    #         text=text
    #     )
    #     response = chat(messages)
    #     recipe = extract_json(response.content)
    # else:
    #     messages = initial_prompt_template.format_messages(
    #         output_format=json.dumps(recipe_format,indent=2,ensure_ascii=False),
    #         text=text,
    #         data=json.dumps(recipe,indent=2,ensure_ascii=False)
    #     )
    #     response = chat(messages)
    #     recipe = extract_json(response.content)
    # print(json.dumps(recipe,indent=2,ensure_ascii=False))

1 ------------------------------
味しみ定番☆  
                肉じゃが

30 分

煮物の定番♪「ほんだし」で煮込んでおいしさ凝縮☆照りとつや感のあるきれいな仕上がり☆

30 分

＃定番・基本

＃栄養バランス

＃肉じゃが

＃牛肉

＃メイン料理

＃和風

おすすめの組み合わせ

このレシピを印刷する

13141
                
                レシピをサイトに保存する

レシピをLINEに保存する

LINEブックマーク登録

LINEに保存とは

おすすめの組み合わせ

材料（4人分）

牛こま切れ肉

150g

玉ねぎ

1個（200g）

にんじん

1本（150g）

じゃがいも

3個（450g）

2カップ

大さじ2

「ほんだし」

小さじ1

砂糖

大さじ1

みりん

大さじ1

しょうゆ

大さじ2

さやいんげん（ゆで）

適量

大さじ1

レシピ提供：味の素KK

このレシピで使われている商品はこちら



つくり方

1玉ねぎは８等分のくし形切りにし、にんじんは１．５ｃｍ幅のいちょう切りにする。じゃがいもは４等分に切る。さやいんげんは３ｃｍ長さに切る。

2フライパンに油を中火で熱し、牛肉を入れて炒める。肉の色が変わったら、（１）の玉ねぎ・にんじん・じゃがいもの順に加え炒め合わせる。

3Ａを加え、煮立ったらアクを取り、フタをして弱火で１０分煮、Ｂを加えてさらに５分煮る。しょうゆを加え、フタをせず５分煮る。フライパンの底から返すように煮汁を煮からめる。

4器に盛り、（１）のさやいんげんを散らす。

栄養情報 （1人分）

・エネルギー334 kcal

・塩分1.7 g

・たんぱく質7.7 g

・野菜摂取量※88 g

※野菜摂取量はきのこ類・いも類を除く

キーワードで探す

# メイン料理

# 和風

# 栄養バランス

# 牛肉

# 定番・基本

# ほんだし®

# 肉じゃが

# じゃがいも

ログインなしで保存できます

気になるレシピを保存して便利に使おう

13141
                
                レシピをサイトに保存する

レシピをLINEに保存する

LINEに保存